In [1]:
import pandas as pd
import pickle
import json
import ast

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
# from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

2025-02-19 20:01:36.614008: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-19 20:01:36.621081: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-19 20:01:36.685810: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-19 20:01:36.746788: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739975496.791214  690610 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739975496.80

In [2]:
ratings_df = pickle.load(open('../backend/processed_data/new_ratings_df.pkl', 'rb'))

In [3]:
ratings_df.head()

,userId,movieId,rating,timestamp,keywords
0,1,110,1.0,1425941529,"[newspaper, adversary, reporter, experience]"
1,1,147,4.5,1425942435,"[odessa, revenge]"
2,1,858,5.0,1425941523,"[letter, arkansas, childbirth, biracial, matri..."
4,1,1246,5.0,1425941556,"[friendship, romantic comedy, teenager, lovesi..."
5,1,1968,4.0,1425942148,"[pop, pop culture, pop star]"


In [4]:
ratings_df.shape

(11436568, 5)

In [5]:
ratings_df.duplicated(subset =  [col for col in ratings_df.columns if ratings_df[col].dtype != 'object']).sum()

np.int64(0)

In [6]:
credits = pd.read_csv('../backend/datasets/credits.csv')
# ratings = pd.read_csv('../backend/datasets/ratings.csv')
# keywords = pd.read_csv('../backend/datasets/keywords.csv')

In [7]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [8]:
credits.shape

(45476, 3)

In [9]:
credits.duplicated().sum()

np.int64(37)

In [10]:
credits = credits.drop_duplicates()

In [11]:
credits.shape

(45439, 3)

    Objective: We have credits and new_ratings_df dataset. new_ratings_df dataset obtained after processing the 'keywords' column in ratings.df file. We will extract the cast and crew from credits whose movie id is listed in new_ratings_df.

In [12]:
for cast in ast.literal_eval(credits['cast'].iloc[0]):
    print(cast)

{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}
{'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}
{'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}
{'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}
{'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tNORKNq

In [13]:
for cast in ast.literal_eval(credits['cast'].iloc[0]):
    print(cast['name'])

Tom Hanks
Tim Allen
Don Rickles
Jim Varney
Wallace Shawn
John Ratzenberger
Annie Potts
John Morris
Erik von Detten
Laurie Metcalf
R. Lee Ermey
Sarah Freeman
Penn Jillette


In [14]:
def extract_name(row):
    return (entry['name'] for entry in ast.literal_eval(row))

In [15]:
credits['cast_name'] = credits['cast'].apply(extract_name)
credits['crew_name'] = credits['crew'].apply(extract_name)

In [16]:
def take_n_name(row, n=10):
    return list(row)[0:n]

In [17]:
# credits['cast_name'] = credits['cast_name'].apply(lambda x: x.split(', '))
# credits['crew_name'] = credits['crew_name'].apply(lambda x: x.split(', '))

credits['keywords'] = credits.apply(
    lambda x: take_n_name(x['cast_name'], 10) + take_n_name(x['crew_name'], 10),
    axis=1
)

In [18]:
credits.head(1)

,cast,crew,id,cast_name,crew_name,keywords
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,<generator object extract_name.<locals>.<genex...,<generator object extract_name.<locals>.<genex...,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."


In [19]:
len(credits['keywords'].iloc[0])

20

In [20]:
ratings_df=ratings_df.merge(credits[['id','keywords']], left_on='movieId', right_on='id', how='inner')

In [21]:
ratings_df.shape

(11437604, 7)

In [22]:
ratings_df.duplicated(subset =  [col for col in ratings_df.columns if ratings_df[col].dtype != 'object']).sum()

np.int64(1036)

In [23]:
ratings_df.drop_duplicates(
    subset=[col for col in ratings_df.columns if ratings_df[col].dtype != 'object'], 
    keep='first', 
    inplace=True
)

In [24]:
11437604 - 1036

11436568

In [25]:
ratings_df.shape

(11436568, 7)

In [26]:
ratings_df = ratings_df[0: 2000000]

In [27]:
ratings_df.shape

(2000000, 7)

In [28]:
ratings_df.head(5)

,userId,movieId,rating,timestamp,keywords_x,id,keywords_y
0,1,110,1.0,1425941529,"[newspaper, adversary, reporter, experience]",110,"[Irène Jacob, Jean-Louis Trintignant, Jean-Pie..."
1,1,147,4.5,1425942435,"[odessa, revenge]",147,"[Jean-Pierre Léaud, Claire Maurier, Albert Rém..."
2,1,858,5.0,1425941523,"[letter, arkansas, childbirth, biracial, matri...",858,"[Tom Hanks, Meg Ryan, Bill Pullman, Ross Malin..."
3,1,1246,5.0,1425941556,"[friendship, romantic comedy, teenager, lovesi...",1246,"[Sylvester Stallone, Burt Young, Milo Ventimig..."
4,1,1968,4.0,1425942148,"[pop, pop culture, pop star]",1968,"[Matthew Perry, Salma Hayek, Jon Tenney, Siobh..."


In [29]:
ratings_df['keywords'] = ratings_df['keywords_x'] + ratings_df['keywords_y']

In [30]:
ratings_df.head(5)

,userId,movieId,rating,timestamp,keywords_x,id,keywords_y,keywords
0,1,110,1.0,1425941529,"[newspaper, adversary, reporter, experience]",110,"[Irène Jacob, Jean-Louis Trintignant, Jean-Pie...","[newspaper, adversary, reporter, experience, I..."
1,1,147,4.5,1425942435,"[odessa, revenge]",147,"[Jean-Pierre Léaud, Claire Maurier, Albert Rém...","[odessa, revenge, Jean-Pierre Léaud, Claire Ma..."
2,1,858,5.0,1425941523,"[letter, arkansas, childbirth, biracial, matri...",858,"[Tom Hanks, Meg Ryan, Bill Pullman, Ross Malin...","[letter, arkansas, childbirth, biracial, matri..."
3,1,1246,5.0,1425941556,"[friendship, romantic comedy, teenager, lovesi...",1246,"[Sylvester Stallone, Burt Young, Milo Ventimig...","[friendship, romantic comedy, teenager, lovesi..."
4,1,1968,4.0,1425942148,"[pop, pop culture, pop star]",1968,"[Matthew Perry, Salma Hayek, Jon Tenney, Siobh...","[pop, pop culture, pop star, Matthew Perry, Sa..."


In [31]:
ratings_df = ratings_df.drop(columns=['keywords_x', 'keywords_y', 'id'])

In [32]:
ratings_df.head()

,userId,movieId,rating,timestamp,keywords
0,1,110,1.0,1425941529,"[newspaper, adversary, reporter, experience, I..."
1,1,147,4.5,1425942435,"[odessa, revenge, Jean-Pierre Léaud, Claire Ma..."
2,1,858,5.0,1425941523,"[letter, arkansas, childbirth, biracial, matri..."
3,1,1246,5.0,1425941556,"[friendship, romantic comedy, teenager, lovesi..."
4,1,1968,4.0,1425942148,"[pop, pop culture, pop star, Matthew Perry, Sa..."


In [33]:
# we will make a neural network which will evaluate on input movieId and output list of userId with possible ratings

In [34]:
# To do this task i first got confused how to pass userId, movieId, ratings and keywords, four things in neural network,
# but I realize we don't need to pass movieId, we can pass keywords and userId in input layer and rating will be
# our output. Based on userId and keywords our neuron will learn rating patterns of the user on given keywords.
# From user we will ask for userId and movieId, from movieId we will extract the keywords, then our model will 
# predict the ouput. 
# Here we will also perform decision tree, random forest and Bayes theorem and compare the accuracy.

In [35]:
# Performing Encoding

user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

ratings_df['userId'] = user_encoder.fit_transform(ratings_df['userId'])
ratings_df['movieId'] = movie_encoder.fit_transform(ratings_df['movieId'])

In [36]:
ratings_df.head()

,userId,movieId,rating,timestamp,keywords
0,0,72,1.0,1425941529,"[newspaper, adversary, reporter, experience, I..."
1,0,104,4.5,1425942435,"[odessa, revenge, Jean-Pierre Léaud, Claire Ma..."
2,0,666,5.0,1425941523,"[letter, arkansas, childbirth, biracial, matri..."
3,0,815,5.0,1425941556,"[friendship, romantic comedy, teenager, lovesi..."
4,0,1206,4.0,1425942148,"[pop, pop culture, pop star, Matthew Perry, Sa..."


In [37]:
# Normalization

scaler = MinMaxScaler()
ratings_df['rating'] = scaler.fit_transform(ratings_df[['rating']])

In [38]:
# Convert keywords to TF-IDF vectors

tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Select top 100 important keywords
keywords_matrix = tfidf_vectorizer.fit_transform(ratings_df['keywords'].astype(str)).toarray()

In [39]:
ratings_df.head()

,userId,movieId,rating,timestamp,keywords
0,0,72,0.111111,1425941529,"[newspaper, adversary, reporter, experience, I..."
1,0,104,0.888889,1425942435,"[odessa, revenge, Jean-Pierre Léaud, Claire Ma..."
2,0,666,1.000000,1425941523,"[letter, arkansas, childbirth, biracial, matri..."
3,0,815,1.000000,1425941556,"[friendship, romantic comedy, teenager, lovesi..."
4,0,1206,0.777778,1425942148,"[pop, pop culture, pop star, Matthew Perry, Sa..."


In [40]:
# Train-test split
train_df, test_df, train_keywords, test_keywords = train_test_split(
    ratings_df, keywords_matrix, test_size=0.2, random_state=42
)

In [41]:
# user tensorflow
# create tensor - data preparation
# import and create Normalizer
# pass the normalizer inside sequentiallayer
# create other layers(Input and dense layer)
# visualize the layers - create summary(model.summary()) and ploting(tf.keras.util.plot_model(model, to_file="", show_shapes=True))
# compile the model and calculate loss for optimization
# check modle performance using metric attribute

In [42]:
num_users = ratings_df['userId'].nunique()
num_movies = ratings_df['movieId'].nunique()
keyword_dim = train_keywords.shape[1]
print(num_users)
print(num_movies)
print(keyword_dim)

46077
6682
100


In [43]:
# creating Input layer

user_input = Input(shape=(1,), name="User_Input")
movie_input = Input(shape=(1,), name="Movie_Input")
keyword_input = Input(shape=(keyword_dim,), name="Keyword_Input")

In [44]:
# Embedding layers

user_embedding = Embedding(input_dim=num_users, output_dim=50, name="User_Embedding")(user_input)
movie_embedding = Embedding(input_dim=num_movies, output_dim=50, name="Movie_Embedding")(movie_input)

2025-02-19 20:02:55.732506: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [45]:
# Flatten/Vectorizing embeddings

user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)

In [46]:
# Concatenate embeddings

concat = Concatenate()([user_vec, movie_vec, keyword_input])

In [47]:
# Deep Neural Network with Dropout and BatchNorm
dense_1 = Dense(256, activation='relu')(concat)
dense_1 = BatchNormalization()(dense_1)
dense_1 = Dropout(0.3)(dense_1)

dense_2 = Dense(128, activation='relu')(dense_1)
dense_2 = BatchNormalization()(dense_2)
dense_2 = Dropout(0.2)(dense_2)

dense_3 = Dense(64, activation='relu')(dense_2)

output = Dense(1, activation='sigmoid')(dense_3)  # Output is normalized

In [48]:
# Model Definition

model = Model(inputs=[user_input, movie_input, keyword_input], outputs=output)
model.summary()
model.compile(loss='mse', optimizer=Adam(learning_rate=0.001), metrics=['mae'])

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ User_Input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Movie_Input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ User_Embedding      │ (None, 1, 50)     │  2,303,850 │ User_Input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Movie_Embedding     │ (None, 1, 50)     │    334,100 │ Movie_Input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 50)        │          0 │ User_Embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 50)        │          0 │ Movie_Embedding[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Keyword_Input       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 200)       │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0],  │
│                     │                   │            │ Keyword_Input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     51,456 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256)       │      1,024 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         65 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,732,159 (10.42 MB)

 Trainable params: 2,731,391 (10.42 MB)

 Non-trainable params: 768 (3.00 KB)

In [49]:
# Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Train Model

import time

start_time = time.time()
model.fit(
    [train_df['userId'], train_df['movieId'], train_keywords],
    train_df['rating'],
    validation_data=([test_df['userId'], test_df['movieId'], test_keywords], test_df['rating']),
    batch_size=64, epochs=8, verbose=1, callbacks=[early_stop]
)
end_time = time.time()

total_time = end_time - start_time

print("total time = {total_time: .4f} seconds")

Epoch 1/8


/home/user/Desktop/internship project/Movie-Recommendator-System/env/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['User_Input', 'Movie_Input', 'Keyword_Input']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


25000/25000 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0449 - mae: 0.1642

/home/user/Desktop/internship project/Movie-Recommendator-System/env/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['User_Input', 'Movie_Input', 'Keyword_Input']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


25000/25000 ━━━━━━━━━━━━━━━━━━━━ 424s 17ms/step - loss: 0.0449 - mae: 0.1642 - val_loss: 0.0377 - val_mae: 0.1497
Epoch 2/8
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 475s 19ms/step - loss: 0.0363 - mae: 0.1462 - val_loss: 0.0363 - val_mae: 0.1465
Epoch 3/8
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 475s 19ms/step - loss: 0.0340 - mae: 0.1408 - val_loss: 0.0357 - val_mae: 0.1445
Epoch 4/8
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 444s 18ms/step - loss: 0.0326 - mae: 0.1373 - val_loss: 0.0357 - val_mae: 0.1448
Epoch 5/8
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 473s 19ms/step - loss: 0.0314 - mae: 0.1344 - val_loss: 0.0354 - val_mae: 0.1433
Epoch 6/8
11464/25000 ━━━━━━━━━━━━━━━━━━━━ 4:20 19ms/step - loss: 0.0301 - mae: 0.1310

In [ ]:
# Save encoders, scaler, and TF-IDF
with open("../backend/user_encoder.pkl", "wb") as f:
    pickle.dump(user_encoder, f)
with open("../backend/movie_encoder.pkl", "wb") as f:
    pickle.dump(movie_encoder, f)
with open("../backend/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
with open("../backend/tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)

In [ ]:
def recommend_top_movies(user_id, top_n=5):
    user_encoded = user_encoder.transform([user_id])[0]
    all_movie_ids = ratings_df['movieId'].unique()

    all_keywords = tfidf_vectorizer.transform(ratings_df['keywords'].astype(str)).toarray()

    predictions = model.predict([np.array([user_encoded] * len(all_movie_ids)), all_movie_ids, all_keywords])

    top_indices = predictions.flatten().argsort()[-top_n:][::-1]
    recommended_movies = movie_encoder.inverse_transform(all_movie_ids[top_indices])
    
    return recommended_movies

user_id = 0
top_movies = recommend_top_movies(user_id)
print("Recommended Movies:", top_movies)

In [ ]:
# Normalization
# ratings_df['rating'] = ratings_df['rating'] / ratings_df['rating'].max()

In [ ]:
# Encoding Categorical Data

# user_encoder = LabelEncoder()
# movie_encoder = LabelEncoder()

# ratings_df['user'] = user_encoder.fit_transform(ratings_df['userId'])
# ratings_df['movie'] = movie_encoder.fit_transform(ratings_df['movieId'])

In [ ]:
# Splitting data into training and testing data

# train, test = train_test_split(ratings_df, test_size=0.2, random_state=42)

In [ ]:
# # Achitecture

# # Constants
# num_users = ratings_df['user'].nunique()
# num_movies = ratings_df['movie'].nunique()
# embedding_dim = 50

# # Inputs
# user_input = Input(shape=(1,), name='user_input')
# movie_input = Input(shape=(1,), name='movie_input')

# # Embeddings
# user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_embedding')(user_input)
# movie_embedding = Embedding(input_dim=num_movies, output_dim=embedding_dim, name='movie_embedding')(movie_input)

# # Flatten embeddings
# user_vec = Flatten()(user_embedding)
# movie_vec = Flatten()(movie_embedding)

# # Concatenate user and movie vectors
# concat = Concatenate()([user_vec, movie_vec])

# # Fully connected layers
# dense_1 = Dense(128, activation='relu')(concat)
# dropout_1 = Dropout(0.5)(dense_1)
# dense_2 = Dense(64, activation='relu')(dropout_1)
# output = Dense(1, activation='sigmoid', name='output')(dense_2)  # Output in range [0, 1]

# # Model
# model = Model(inputs=[user_input, movie_input], outputs=output)
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model.summary()


In [ ]:
# # Training the model

# history = model.fit(
#     [train['user'], train['movie']],
#     train['rating'],
#     epochs=1,
#     batch_size=64,
#     validation_split=0.2
# )

In [ ]:
# # Generate recommendation

# def recommend_movies(user_id, model, movie_ids, user_encoder, movie_encoder):
#     user_idx = user_encoder.transform([user_id])[0]
#     movie_indices = movie_encoder.transform(movie_ids)
#     predictions = model.predict([np.array([user_idx] * len(movie_indices)), np.array(movie_indices)])
#     top_movies = np.argsort(predictions[:, 0])[-10:][::-1]  # Top 10 recommendations
#     return movie_ids[top_movies]

# # Example usage
# user_id = 1
# movie_ids = ratings_df['movieId'].unique()
# recommended_movies = recommend_movies(user_id, model, movie_ids, user_encoder, movie_encoder)
# print("Recommended Movies:", recommended_movies)


In [ ]:
# # Evaluation

# y_pred = model.predict([test['user'], test['movie']])
# mse = mean_squared_error(test['rating'], y_pred)
# print(f"Test MSE: {mse}")
